In [1]:
import torch
import torch.nn as nn
from torch.nn import Parameter
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
use_cuda = torch.cuda.is_available()
torch.cuda.set_device(1)
import sys, random
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle
import math
from sklearn.metrics import roc_auc_score

In [2]:
train_sl= pickle.load(open('/data/projects/py_ehr_2/Data/Readm_h143_cl2_mde_tp_S.train', 'rb'), encoding='bytes')
test_sl= pickle.load(open('/data/projects/py_ehr_2/Data/Readm_h143_cl2_mde_tp_S.test', 'rb'), encoding='bytes')
valid_sl= pickle.load(open('/data/projects/py_ehr_2/Data/Readm_h143_cl2_mde_tp_S.valid', 'rb'), encoding='bytes')
print (len(train_sl),len(valid_sl),len(test_sl))

12586 1798 3596


In [3]:
print(train_sl[0])

[13388336, 0, [[[0], [[6244, 2879, 265], [533, 92, 131, 509, 128, 68, 83, 30, 28, 58, 166, 8, 87, 88, 40, 101, 20, 113, 37, 120, 109, 54, 4, 44, 94, 17], []]], [[52], [[58, 2261], [], []]], [[7], [[397, 443, 28, 522, 18, 175, 120, 176, 516], [88, 37, 87, 40, 166, 94, 53, 92, 7, 30, 83, 824, 113, 131, 28], []]], [[9], [[443, 28, 3, 56], [], []]], [[27], [[443], [], []]], [[64], [[19, 132, 44, 610, 328, 443, 228, 671, 397, 3], [], []]], [[25], [[443], [], []]], [[70], [[28, 18, 443], [], []]], [[15], [[397, 218, 163, 18, 93, 1184, 1253, 31, 19, 231, 516, 632, 58], [7], []]], [[14], [[930, 2789, 1453], [], []]], [[13], [[443, 163], [], []]], [[52], [[886, 58, 163], [], []]]]]


In [4]:
class EHR_RNN(nn.Module):
    def __init__(self, input_size,embed_dim, hidden_size, n_layers=1,dropout_r=0.1,cell_type='LSTM',bi=False ,time=False, preTrainEmb=''):
        ### need either to split the input_size, make it a list or a dictionary or take it out, the same for pretrained emb 
        super(EHR_RNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb=preTrainEmb
        self.time=time
        
        if bi: self.bi=2 
        else: self.bi=1
        
        if self.time: self.in_size= (1*embed_dim)+1 
        else: self.in_size=1*embed_dim
            
       # if len(self.preTrainEmb)>0:
       #     emb_t= torch.FloatTensor(np.asmatrix(self.preTrainEmb))
       #     self.embed= nn.Embedding.from_pretrained(emb_t)#,freeze=False)  
       # else:
       #    self.embed= nn.Embedding(input_size, self.embed_dim,padding_idx=0)#,scale_grad_by_freq=True)
##########Replace above with separate embedding for each data category
        self.embed_d= nn.Embedding(8655, self.embed_dim,padding_idx=0)#,scale_grad_by_freq=True)
        self.embed_m= nn.Embedding(925, self.embed_dim,padding_idx=0)#,scale_grad_by_freq=True)
        self.embed_o= nn.Embedding(3410, self.embed_dim,padding_idx=0)#,scale_grad_by_freq=True)
    
        if self.cell_type == "GRU":
            cell = nn.GRU
        elif self.cell_type == "RNN":
            cell = nn.RNN
        elif self.cell_type == "LSTM":
            cell = nn.LSTM
        else:
            raise NotImplementedError
            
          
        self.rnn_c = cell(self.in_size, hidden_size,num_layers=n_layers, dropout= dropout_r , bidirectional=bi , batch_first=True)
        self.out = nn.Linear(self.hidden_size*self.bi,1)
        self.sigmoid = nn.Sigmoid()

        
    def EmbedPatient_MB(self, input): # x is a ehr_seq_tensor

        if use_cuda:
            flt_typ=torch.cuda.FloatTensor
            lnt_typ=torch.cuda.LongTensor
        else: 
            lnt_typ=torch.LongTensor
            flt_typ=torch.FloatTensor
        #mb=[] #split to diff types
        mbd=[]
        mbm=[]
        mbo=[]
        
        mtd=[]
        lbt=[]
        seq_l=[]
        self.bsize=len(input) ## no of pts in minibatch
        lp= len(max(input, key=lambda xmb: len(xmb[-1]))[-1]) ## maximum number of visits per patients in minibatch # this remains fine with whatever input format
        
        #llv=0 # split as well
        llvd=0
        llvm=0
        llvo=0

        for x in input:
                lvd= len(max(x[-1], key=lambda xmb: len(xmb[1][0]))[1][0])
                lvm= len(max(x[-1], key=lambda xmb: len(xmb[1][1]))[1][1])
                lvo= len(max(x[-1], key=lambda xmb: len(xmb[1][2]))[1][2])
                #print(lvd,lvm,lvo)
                if llvd < lvd:  llvd=lvd     # max number of diagnosis codes per visit in minibatch        
                if llvm < lvm:  llvm=lvm     # max number of medication codes per visit in minibatch        
                if llvo < lvo:  llvo=lvo     # max number of demographics and other codes per visit in minibatch        

        #print(llvd,llvm,llvo)        
        #if llvd==0: llvd=1
        #if llvm==0: llvm=1
        
            
        '''
        llv=0
        for x in input:
            lv= len(max(x[-1], key=lambda xmb: len(xmb[1]))[1])
            if llv < lv:
                llv=lv     # max number of codes per visit in minibatch        
        '''
        for pt in input:
            sk,label,ehr_seq_l = pt
            lpx=len(ehr_seq_l) ## no of visits in pt record
            seq_l.append(lpx) 
            lbt.append(Variable(flt_typ([[float(label)]])))### check if code issue replace back to the above
            
            #ml=(len(max(ehr_seq_l, key=len))) # max number of codes per visit in pt record
            time_dim=[]
            #ehr_seq_tl=[]  ## split to diff types        
            ehr_seq_tld=[]
            ehr_seq_tlm=[]
            ehr_seq_tlo=[]
          
            for ehr_seq in ehr_seq_l:
                #pd=(0, (llv -len(ehr_seq[1])))
                #result = F.pad(torch.from_numpy(np.asarray(ehr_seq[1],dtype=int)).type(lnt_typ),pd,"constant", 0)
                #ehr_seq_tl.append(result)    ## split to diff types 
                pdd=(0, (llvd -len(ehr_seq[1][0])))
                pdm=(0, (llvm -len(ehr_seq[1][1])))
                pdo=(0, (llvo -len(ehr_seq[1][2])))
                #print(pdd,pdm,pdo)
                if len(ehr_seq[1][0])==0: resultd = F.pad(lnt_typ([0]),(0,llvd-1),"constant", 0)    
                else: resultd = F.pad(torch.from_numpy(np.asarray(ehr_seq[1][0],dtype=int)).type(lnt_typ),pdd,"constant", 0)
                if len(ehr_seq[1][1])==0: resultm = F.pad(lnt_typ([0]),(0,llvm-1),"constant", 0)     
                else:resultm = F.pad(torch.from_numpy(np.asarray(ehr_seq[1][1],dtype=int)).type(lnt_typ),pdm,"constant", 0)
                if len(ehr_seq[1][2])==0: resulto = F.pad(lnt_typ([0]),(0,llvo-1),"constant", 0)     
                else: resulto = F.pad(torch.from_numpy(np.asarray(ehr_seq[1][2],dtype=int)).type(lnt_typ),pdo,"constant", 0)
                ehr_seq_tld.append(resultd)
                ehr_seq_tlm.append(resultm)
                ehr_seq_tlo.append(resulto)

                
                if self.time:                 
                    #time_dim.append(Variable(torch.from_numpy(np.asarray(ehr_seq[0],dtype=int)).type(flt_typ)))
                    # use log time as RETAIN
                    time_dim.append(Variable(torch.div(flt_typ([1.0]), torch.log(torch.from_numpy(np.asarray(ehr_seq[0],dtype=int)).type(flt_typ) + flt_typ([2.7183])))))
                    
            lpp= lp-lpx ## diff be max seq in minibatch and cnt of pt visits
            zp= nn.ZeroPad2d((0,0,0,lpp)) ## (0,0,0,lpp) when use the pack padded seq and (0,0,lpp,0) otherwise
  
            #ehr_seq_t= Variable(torch.stack(ehr_seq_tl,0)) 
            #ehr_seq_t= zp(ehr_seq_t) ## zero pad the visits med codes  
            #mb.append(ehr_seq_t) ## split to diff types
            ehr_seq_td= Variable(torch.stack(ehr_seq_tld,0)) 
            ehr_seq_tm= Variable(torch.stack(ehr_seq_tlm,0)) 
            ehr_seq_to= Variable(torch.stack(ehr_seq_tlo,0)) 
            ehr_seq_td= zp(ehr_seq_td) ## zero pad the visits diag codes
            ehr_seq_tm= zp(ehr_seq_tm) ## zero pad the visits med codes
            ehr_seq_to= zp(ehr_seq_to) ## zero pad the visits dem&other codes
            mbd.append(ehr_seq_td)
            mbm.append(ehr_seq_tm)
            mbo.append(ehr_seq_to)
            
            if self.time:
                time_dim_v= Variable(torch.stack(time_dim,0))
                time_dim_pv= zp(time_dim_v)## zero pad the visits time diff codes
                mtd.append(time_dim_pv)

            
        #mb_t= Variable(torch.stack(mb,0)) 
        #if use_cuda:
        #    mb_t.cuda()
        #embedded = self.embed(mb_t)  ## Embedding for codes
        #embedded = torch.sum(embedded, dim=2) #### split all the above
        
        mb_td= Variable(torch.stack(mbd,0))
        mb_tm= Variable(torch.stack(mbm,0))
        mb_to= Variable(torch.stack(mbo,0))
        if use_cuda:
            mb_td.cuda()
            mb_tm.cuda()
            mb_to.cuda()
        embedded_d = torch.sum(self.embed_d(mb_td), dim=2) 
        embedded_m = torch.sum(self.embed_m(mb_tm), dim=2) 
        embedded_o = torch.sum(self.embed_o(mb_to), dim=2)      

        #embedded=torch.cat((embedded_d,embedded_m,embedded_o),dim=2)## the concatination of above
        #embedded=torch.cat((embedded_d,embedded_m),dim=2)## the concatination of above
        embedded=embedded_d
        lbt_t= Variable(torch.stack(lbt,0))
        if self.time:
            mtd_t= Variable(torch.stack(mtd,0))
            if use_cuda: mtd_t.cuda()
            out_emb= torch.cat((embedded,mtd_t),dim=2)
        else:
            out_emb= embedded
        return out_emb, lbt_t,seq_l #,dem_emb
    
    def init_hidden(self):
        
        h_0 = Variable(torch.rand(self.n_layers*self.bi,self.bsize, self.hidden_size))
        
        if self.cell_type == "LSTM":
            result = (h_0,h_0)
        else: 
            result = h_0
            
        if use_cuda:
            return result.cuda()
        else:
            return result
    
    def forward(self, input):
        
        x_in , lt ,x_lens = self.EmbedPatient_MB(input)
        x_inp = nn.utils.rnn.pack_padded_sequence(x_in,x_lens,batch_first=True)   
        #h_0= self.init_hidden()
        output, hidden = self.rnn_c(x_inp)#,h_0) 
        if self.cell_type == "LSTM":
            hidden=hidden[0]
        if self.bi==2:
            output = self.sigmoid(self.out(torch.cat((hidden[-2],hidden[-1]),1)))
        else:
            output = self.sigmoid(self.out(hidden[-1]))
        return output.squeeze(), lt.squeeze()


In [5]:
def train (tmodel,mini_batch, criterion, optimizer):  
    tmodel.zero_grad()
    output , label_tensor = tmodel(mini_batch)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
   
    return output, loss.item()

In [6]:
def run_model_train(tmodel,dataset,batch_size,optimizer):
        
    tmodel.train()
    dataset.sort(key=lambda pt:len(pt[-1]),reverse=True) 
    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    print_every = 10#int(batch_size/2)
    plot_every = 5
    iter=0
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    start = time.time()

    for index in random.sample(range(n_batches), n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, loss = train(tmodel,batch, criterion = nn.BCELoss(), optimizer = optimizer)
            current_loss += loss
            iter +=1
            # Add current loss avg to list of losses
            if iter % plot_every == 0:
                all_losses.append(current_loss / plot_every)
                current_loss = 0
                
    return current_loss,all_losses


In [7]:
def calculate_auc(test_model, dataset, batch_size=200):
    test_model.eval()
    dataset.sort(key=lambda pt:len(pt[-1]),reverse=True) 
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    labelVec =[]
    y_hat= []
    
    for index in range(n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, label_t = test_model(batch)
            y_hat.extend(output.cpu().data.view(-1).numpy())
            labelVec.extend(label_t.cpu().data.view(-1).numpy())
    auc = roc_auc_score(labelVec, y_hat)
    
    return auc

In [8]:
# training all samples in random order
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
epochs=100
batch_size=100
## Optim param
learning_rate = 0.01
l2=1e-04
epsl=1e-06 
## for final results reporting
current_loss_l=[]
all_losses_l=[]
train_auc_allep =[]
valid_auc_allep =[]
test_auc_allep=[]
bestValidAuc = 0.0
bestTestAuc = 0.0
bestValidEpoch = 0

### Define Model Parameters
model = EHR_RNN(input_size=15816, hidden_size=64 ,embed_dim=64, dropout_r=0, cell_type='GRU', n_layers=1,bi=False ,time=False, preTrainEmb='')
if use_cuda:
    model = model.cuda()
    
### Select Optimizer
####################
#optimizer = optim.SGD(tmodel.parameters(), lr=learning_rate)#, weight_decay=l2)
#optimizer = optim.Adadelta(tmodel.parameters(), lr=learning_rate, weight_decay=l2)
#optimizer = optim.ASGD(tmodel.parameters(), lr=learning_rate, weight_decay=l2 )
#optimizer = optim.SparseAdam (tmodel.parameters(),lr=learning_rate) #'''lr=learning_rate,''' 
#optimizer = optim.Adagrad (tmodel.parameters(),lr=learning_rate, weight_decay=l2) #'''lr=learning_rate,''' 
optimizer = optim.Adamax(model.parameters(), lr=learning_rate, weight_decay=l2 ,eps=epsl)
#optimizer = optim.Adamax(filter(lambda p: p.requires_grad, tmodel.parameters()), lr=learning_rate, weight_decay=l2 ,eps=epsl) ### Beta defaults (0.9, 0.999)
#optimizer = optim.RMSprop (tmodel.parameters(),lr=learning_rate, weight_decay=l2 ,eps=epsl)
#optimizer = optim.Adam(tmodel.parameters(), lr=learning_rate, weight_decay=learning_rate)
    
### Run Epochs    
for ep in range(epochs):
    
    #print (model.embed.weight.data[135] ) ## checkpoint  for embedding
    #print (model.state_dict() )## checkpoint  for all learnable parameters
    start = time.time()
    current_loss_la,all_losses_la = run_model_train(model,train_sl,batch_size,optimizer)
    train_time = timeSince(start)
    eval_start = time.time()
    train_auc = calculate_auc(model,train_sl,batch_size)
    test_auc = calculate_auc(model,test_sl,batch_size)
    valid_auc = calculate_auc(model,valid_sl,batch_size)
    eval_time = timeSince(eval_start)
    all_losses_l.append (all_losses_la)
    avg_loss = np.mean(all_losses_la)
    train_auc_allep.append(train_auc)
    valid_auc_allep.append(valid_auc)
    test_auc_allep.append(test_auc)
    current_loss_l.append(current_loss_la)
    print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc," Avg Loss: ", avg_loss, 'Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
     
    if valid_auc > bestValidAuc: 
        bestValidAuc = valid_auc
        bestValidEpoch = ep
        bestTestAuc = test_auc
        best_model = model
        #torch.save(best_model, bmodel_pth)
        #torch.save(best_model.state_dict(), bmodel_st)
    if ep - bestValidEpoch >12: break
            
print ('bestValidAuc %f has a TestAuc of %f at epoch %d ' % (bestValidAuc, bestTestAuc, bestValidEpoch))


# for results comparison
# Baseline GRU Epoch  6  Train_auc : 0.8908693216988484  , Valid_auc :  0.8325830431077114  ,& Test_auc :  0.8366408413283413  Avg Loss:  0.2512624728982732 Train Time (1m 0s) Eval Time (1m 24s)
# Same as above using pack padded seq Epoch  6  Train_auc : 0.8872790431179439  , Valid_auc :  0.8216659126061384  ,& Test_auc :  0.8356468887718888  Avg Loss:  0.24958992269584687 Train Time (0m 54s) Eval Time (1m 18s)
# Baseline GRU with time Epoch  3  Train_auc : 0.8761884834388459  , Valid_auc :  0.8256302543112971  ,& Test_auc :  0.8427002895752896  Avg Loss:  0.2645566132109044 Train Time (0m 56s) Eval Time (1m 20s)
## Bi-directional GRU with time using log time Epoch  4  Train_auc : 0.8847842143005156  , Valid_auc :  0.841847991919277  ,& Test_auc :  0.8389933358683359  Avg Loss:  0.256076513887462 Train Time (2m 8s) Eval Time (3m 5s)


Epoch  0  Train_auc : 0.8327374484550133  , Valid_auc :  0.7801049393201593  ,& Test_auc :  0.7790686982792592  Avg Loss:  0.6071783783435821 Train Time (1m 20s) Eval Time (1m 55s)
Epoch  1  Train_auc : 0.8854537905792891  , Valid_auc :  0.8154181529924693  ,& Test_auc :  0.8116567525473344  Avg Loss:  0.5004394273757935 Train Time (1m 21s) Eval Time (1m 58s)
Epoch  2  Train_auc : 0.9286089853066386  , Valid_auc :  0.8353050349483168  ,& Test_auc :  0.8319824172937269  Avg Loss:  0.4235741053819656 Train Time (1m 19s) Eval Time (1m 55s)


In [ ]:
print(model)